## Loading the dataset

In [ ]:
import pandas as pd

# Load your dataset
df = pd.read_csv("/content/train.csv")
df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50


## a)handling missing values

In [ ]:
# Drop 'New_Price' if >80% missing
if 'New_Price' in df.columns and df['New_Price'].isnull().mean() > 0.8:
    df.drop(columns=['New_Price'], inplace=True)

# Impute missing values
for col in df.columns:
    if df[col].isnull().sum() > 0:
        if df[col].dtype in ['float64', 'int64']:
            df[col] = df[col].fillna(df[col].median())
        else:
            df[col] = df[col].fillna(df[col].mode()[0])

# Display first 10 rows
print(df.head().to_string(index=False))


 Unnamed: 0                             Name   Location  Year  Kilometers_Driven Fuel_Type Transmission Owner_Type    Mileage  Engine     Power  Seats  Price
          1 Hyundai Creta 1.6 CRDi SX Option       Pune  2015              41000    Diesel       Manual      First 19.67 kmpl 1582 CC 126.2 bhp    5.0  12.50
          2                     Honda Jazz V    Chennai  2011              46000    Petrol       Manual      First   13 km/kg 1199 CC  88.7 bhp    5.0   4.50
          3                Maruti Ertiga VDI    Chennai  2012              87000    Diesel       Manual      First 20.77 kmpl 1248 CC 88.76 bhp    7.0   6.00
          4  Audi A4 New 2.0 TDI Multitronic Coimbatore  2013              40670    Diesel    Automatic     Second  15.2 kmpl 1968 CC 140.8 bhp    5.0  17.74
          6           Nissan Micra Diesel XV     Jaipur  2013              86999    Diesel       Manual      First 23.08 kmpl 1461 CC  63.1 bhp    5.0   3.50


## Justification for Data Cleaning and Preprocessing
Due to its over 80% missing values, the New_Price field was removed since it was not appropriate for trustworthy imputation or analysis.

To reduce the impact of outliers and maintain central tendency, missing values in numerical features (such Seats, Power, or Engine) for the remaining columns were imputed using the median.

The mode was used to impute missing values in categorical columns (like Owner_Type), making sure that the most common category was selected to preserve logical consistency.

This approach guarantees that the dataset is as clean and consistent as feasible for subsequent analysis, while simultaneously preserving as much useful information as possible.

## b)removing units from columns

In [ ]:

# Define function to extract numeric part
def extract_number(x):
    if pd.isnull(x):
        return np.nan
    return float(str(x).split()[0].replace(',', ''))

# Clean columns if they exist
if 'Mileage' in df.columns:
    df['Mileage'] = df['Mileage'].apply(extract_number)

if 'Engine' in df.columns:
    df['Engine'] = df['Engine'].apply(extract_number)

if 'Power' in df.columns:
    df['Power'] = df['Power'].apply(lambda x: extract_number(str(x).replace('bhp', '')))

if 'New_Price' in df.columns:
    df['New_Price'] = df['New_Price'].astype(str).str.replace('Lakh', '', regex=False)
    df['New_Price'] = df['New_Price'].apply(lambda x: float(x) if x.replace('.', '', 1).isdigit() else np.nan)

# Display first 5 rows of the full dataset in original format with cleaned values
print(df.head().to_string(index=False))


 Unnamed: 0                             Name   Location  Year  Kilometers_Driven Fuel_Type Transmission Owner_Type  Mileage  Engine  Power  Seats  Price
          1 Hyundai Creta 1.6 CRDi SX Option       Pune  2015              41000    Diesel       Manual      First    19.67  1582.0 126.20    5.0  12.50
          2                     Honda Jazz V    Chennai  2011              46000    Petrol       Manual      First    13.00  1199.0  88.70    5.0   4.50
          3                Maruti Ertiga VDI    Chennai  2012              87000    Diesel       Manual      First    20.77  1248.0  88.76    7.0   6.00
          4  Audi A4 New 2.0 TDI Multitronic Coimbatore  2013              40670    Diesel    Automatic     Second    15.20  1968.0 140.80    5.0  17.74
          6           Nissan Micra Diesel XV     Jaipur  2013              86999    Diesel       Manual      First    23.08  1461.0  63.10    5.0   3.50


## c)one hot encode fueltype and transmission

In [ ]:
df_encoded = pd.get_dummies(df, columns=['Fuel_Type', 'Transmission'], drop_first=False)

# Convert any boolean columns to integers (0/1), just in case
for col in df_encoded.columns:
    if df_encoded[col].dtype == bool:
        df_encoded[col] = df_encoded[col].astype(int)

# Display first 5 rows of the encoded columns only
encoded_columns = [col for col in df_encoded.columns if 'Fuel_Type_' in col or 'Transmission_' in col]
print(df_encoded[encoded_columns].head(5).to_string(index=False))

 Fuel_Type_Diesel  Fuel_Type_Electric  Fuel_Type_Petrol  Transmission_Automatic  Transmission_Manual
                1                   0                 0                       0                    1
                0                   0                 1                       0                    1
                1                   0                 0                       0                    1
                1                   0                 0                       1                    0
                1                   0                 0                       0                    1


## d) Creating new feature-car age

In [ ]:
from datetime import datetime
current_year = datetime.now().year
df['Car_Age'] = current_year - df['Year']
#print(df['Car_Age'])
print(df[['Name', 'Year', 'Car_Age']].head())


                               Name  Year  Car_Age
0  Hyundai Creta 1.6 CRDi SX Option  2015       10
1                      Honda Jazz V  2011       14
2                 Maruti Ertiga VDI  2012       13
3   Audi A4 New 2.0 TDI Multitronic  2013       12
4            Nissan Micra Diesel XV  2013       12


## e) Performing select,filter,rename,mutate,arrange,summarize operations

In [ ]:
# Split Name into Make and Model (fix applied)
df[['Make', 'Model']] = df['Name'].str.split(' ', n=1, expand=True)


# Identify encoded columns for Fuel and Transmission
encoded_fuel = [col for col in df.columns if 'Fuel_Type' in col]
encoded_trans = [col for col in df.columns if 'Transmission' in col]

# Select relevant columns
selected_cols = ['Make', 'Model', 'Price', 'Car_Age'] + encoded_fuel + encoded_trans
selected_df = df[selected_cols]

# Filter cars older than 10 years and price > 5 lakh
filtered_df = selected_df[(df['Car_Age'] > 10) & (df['Price'] > 5)]

# Rename 'Price' to 'Used_Price'
renamed_df = filtered_df.rename(columns={'Price': 'Used_Price'})

# Add new column: Price per Year
renamed_df['Price_per_Year'] = renamed_df['Used_Price'] / renamed_df['Car_Age']

# Arrange by price descending
arranged_df = renamed_df.sort_values(by='Used_Price', ascending=False)

# Summarize: group by Make
summary = df.groupby('Make').agg(
    Avg_Price=('Price', 'mean'),
    Count=('Price', 'count')
).reset_index()

# Display sample outputs
print("Top 5 sorted cars:\n", arranged_df.head())
print("\nGrouped Summary by Make:\n", summary.head())


Top 5 sorted cars:
                Make                                       Model  Used_Price  \
5620    Lamborghini                              Gallardo Coupe       120.0   
4556  Mercedes-Benz                            SLK-Class 55 AMG        90.0   
3383        Porsche                             Panamera Diesel        75.0   
5448        Porsche                   Panamera 2010 2013 Diesel        72.0   
1307           Land  Rover Range Rover 3.6 TDV8 Vogue SE Diesel        70.0   

      Car_Age Fuel_Type Transmission  Price_per_Year  
5620       14    Petrol    Automatic        8.571429  
4556       11    Petrol    Automatic        8.181818  
3383       12    Diesel    Automatic        6.250000  
5448       12    Diesel    Automatic        6.000000  
1307       13    Diesel    Automatic        5.384615  

Grouped Summary by Make:
          Make  Avg_Price  Count
0  Ambassador   1.350000      1
1        Audi  25.569787    235
2         BMW  25.243146    267
3     Bentley  59.00